<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3

db = 'site.db'


def create_db():
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY,
                    login TEXT,
                    password TEXT,
                    age INTEGER,
                    city TEXT
                );
            """
        )
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS chats (
                    id INTEGER PRIMARY KEY,
                    user_id TEXT,
                    chatname TEXT,
                    description TEXT,
                    created_at DATETIME
                );
            """
        )
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS chat_users (
                    user_id TEXT,
                    chat_id TEXT,
                    FOREIGN KEY (user_id) REFERENCES users (id),
                    FOREIGN KEY (chat_id) REFERENCES chats (id)
                );
            """
        )
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS messages (
                    id INTEGER PRIMARY KEY,
                    user_id TEXT,
                    chat_id TEXT,
                    message TEXT,
                    created_at DATETIME,
                    FOREIGN KEY (user_id) REFERENCES users (id),
                    FOREIGN KEY (chat_id) REFERENCES chats (id)
                );
            """
        )
        connection.commit()


def register_new_user(login: str, password: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute("SELECT id FROM users WHERE login = ?", (login,))
        user = cursor.fetchone()
        if user:
            return False
        else:
            cursor.execute("INSERT INTO users (login, password) VALUES (?, ?)", (login, password))
            connection.commit()
            return True


def check_user(login: str, password: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute("SELECT id FROM users WHERE login = ? AND password = ?", (login, password))
        user = cursor.fetchone()
        if user:
            return True
        else:
            return False


def load_all_user_chats(login: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    SELECT chats.chatname
                    FROM chats
                    JOIN chat_users ON chats.id = chat_users.chat_id
                    JOIN users ON chat_users.user_id = users.id
                    WHERE users.login = ?;
                """
        cursor.execute(query, (login,))
        return [row[0] for row in cursor.fetchall()]


def create_chat(login: str, chatname: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute("SELECT id FROM chats WHERE chatname=?", (chatname,))
        chat = cursor.fetchone()
        if chat:
            return False
        else:
            query = """
                        INSERT INTO chats (user_id, chatname, created_at)
                        VALUES ((SELECT u.id AS user_id
                                FROM users u
                                WHERE u.login = ?), ?, CURRENT_TIMESTAMP)
                    """
            cursor.execute(query, (login, chatname))
            connection.commit()
            return True


def add_user_to_chat(login: str, chatname: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    INSERT INTO chat_users (user_id, chat_id)
                    VALUES ((SELECT u.id FROM users u WHERE u.login = ?),
                            (SELECT c.id FROM chats c WHERE c.chatname = ?));
                """
        cursor.execute(query, (login, chatname))
        connection.commit()


def find_chat_by_name(chatname: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute("SELECT id FROM chats WHERE chatname=?", (chatname,))
        chat = cursor.fetchone()
        if chat:
            return True
        else:
            return False


def load_all_messages_by_chat_name(chatname: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    SELECT
                        u.login,
                        m.message
                    FROM messages m
                    JOIN chats c ON m.chat_id = c.id
                    JOIN users u ON m.user_id = u.id
                    WHERE c.chatname = ?
                    ORDER BY m.created_at DESC;
                """
        cursor.execute(query, (chatname,))
        return cursor.fetchall()


def send_message(login: str, chatname: str, message: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    INSERT INTO messages (user_id, chat_id, message, created_at)
                    VALUES ((SELECT u.id FROM users u WHERE u.login = ?),
                            (SELECT c.id FROM chats c WHERE c.chatname = ?),
                            ?,
                            CURRENT_TIMESTAMP);
                """
        cursor.execute(query, (login, chatname, message))
        connection.commit()


def delete_user(login: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    DELETE FROM users
                    WHERE login = ?
                """
        cursor.execute(query, (login,))
        connection.commit()


def delete_chat(login: str, chatname: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    DELETE FROM chat_users
                    WHERE user_id = (SELECT u.id FROM users u WHERE u.login = ?)
                    AND chat_id = (SELECT c.id FROM chats c WHERE c.chatname = ?);
                """
        cursor.execute(query, (login, chatname))
        connection.commit()


def delete_message(login: str, chatname: str, message: str):
    with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        query = """
                    DELETE FROM messages
                    WHERE user_id = (SELECT u.id FROM users u WHERE u.login = ?),
                        chat_id = (SELECT c.id FROM chats c WHERE c.chatname = ?),
                        message = ?;
                """
        cursor.execute(query, (login, chatname, message))
        connection.commit()